In [1]:
import torch
from bert import *
from feature_data import *

We want to evaluate the model on its ability to predict features in context. First, we obtain gold vectors for both senses of 10 ambiguous words from the McRae et al. data

TODO: Ensure that the models are not trained on this data. Maybe train on everything but these words?

What are the words?

In [169]:


ambiguous_pairs = [
    ('bat_animal', 'bat_baseball'),
    ('board_wood', 'board_black'),
    ('bow_ribbon', 'bow_weapon'),
    ('cap_bottle', 'cap_hat'),
    #('crane_machine', 'crane_animal')
    ('hose', 'hose_leggings'),
    ('mink', 'mink_coat'), # this one is not fully disambiguated
    ('mouse', 'mouse_computer'),
    ('pipe_smoking', 'pipe_plumbing'),
    ('tank_army', 'tank_container')
]

In [27]:
exs = read_cue_feature_examples('data/buchanan/cue_feature_words.csv')
norms = FeatureNorms(exs)


In [28]:
norms.print_features('bat')
norms.print_features('bat_baseball')
norms.print_features('bat_animal')

for (w1, w2) in ambiguous_pairs:
    norms.print_features(w1)
    norms.print_features(w2)

features for bat: {'animal': 6.779661017, 'cricket': 7.344632768, 'fly': 7.90960452, 'hit': 7.344632768, 'wood': 9.039548023}
features for bat_baseball: {'ball': 60.0, 'heave': 16.66666667, 'hit': 96.66666667, 'long': 50.0, 'metal': 53.33333333, 'sport': 23.33333333, 'swing': 20.0, 'wood': 73.33333333}
features for bat_animal: {'animal': 33.33333333, 'black': 53.33333333, 'blind': 33.33333333, 'cave': 46.66666667, 'down': 33.33333333, 'fang': 23.33333333, 'fly': 63.33333333, 'fur': 36.66666667, 'mammal': 23.33333333, 'navigate': 30.0, 'nocturnal': 63.33333333, 'radar': 30.0, 'scare': 20.0, 'screech': 26.66666667, 'sleep': 33.33333333, 'small': 20.0, 'vampire': 16.66666667, 'wing': 86.66666667}
features for bat_animal: {'animal': 33.33333333, 'black': 53.33333333, 'blind': 33.33333333, 'cave': 46.66666667, 'down': 33.33333333, 'fang': 23.33333333, 'fly': 63.33333333, 'fur': 36.66666667, 'mammal': 23.33333333, 'navigate': 30.0, 'nocturnal': 63.33333333, 'radar': 30.0, 'scare': 20.0, 'scr

Now we want to find examples of each sense and make predictions using the model for that word. We will score the predictions of each example and then average them. Here are 10 examples of bat_animal. Load these from the file we have created with these words.

In [166]:
from nltk.corpus import wordnet as wn

senses_to_analyze = {
    'bat_animal': wn.lemma('bat.n.01.bat'),
    'bat_baseball': wn.lemma('bat.n.05.bat'),
    'board_wood': wn.lemma('board.n.02.board'),
    'board_black': wn.lemma('blackboard.n.01.blackboard'),
    'bow_ribbon': wn.lemma('bow.n.08.bow'),
    'bow_weapon': wn.lemma('bow.n.04.bow'),
    'cap_bottle': wn.lemma('cap.n.02.cap'),
    'cap_hat': wn.lemma('cap.n.01.cap'),
    #('crane_machine', 'crane_animal')
    'hose': wn.lemma('hose.n.03.hose'),
    'hose_leggings': wn.lemma('hose.n.01.hose'),
    'mink': wn.lemma('mink.n.03.mink'),
    'mink_coat': wn.lemma('mink.n.01.mink'), # # this one is not fully disambiguated
    'mouse': wn.lemma('mouse.n.01.mouse'),
    'mouse_computer': wn.lemma('mouse.n.04.mouse'),
    'pipe_smoking':  wn.lemma('pipe.n.01.pipe'),
    'pipe_plumbing': wn.lemma('pipe.n.02.pipe'),
    'tank_army': wn.lemma('tank.n.01.tank'),
    'tank_container': wn.lemma('tank.n.02.tank')
}



for k, v in senses_to_analyze.items():
    print(v.synset())
    

Synset('bat.n.01')
Synset('bat.n.05')
Synset('board.n.02')
Synset('blackboard.n.01')
Synset('bow.n.08')
Synset('bow.n.04')
Synset('cap.n.02')
Synset('cap.n.01')
Synset('hose.n.03')
Synset('hosiery.n.01')
Synset('mink.n.03')
Synset('mink.n.01')
Synset('mouse.n.01')
Synset('mouse.n.04')
Synset('pipe.n.01')
Synset('pipe.n.02')
Synset('tank.n.01')
Synset('tank.n.02')
Synset('bat.n.01')


In [158]:
from nltk.corpus import semcor
from nltk.tree import Tree
import itertools
import random


wordnet_senses = [
                #wn.lemma('produce.v.04.produce'),
                 wn.lemma('bat.n.01.bat'),
                 wn.lemma('bat.n.05.bat'),
                 wn.lemma('mouse.n.04.mouse')
                ]


def traverse_tree(tree):
    # print("tree:", tree)
    for subtree in tree:
        if type(subtree) == Tree:
            print(subtree.label())
            traverse_tree(subtree)


def find_tokens(wn_lemma, sents):

    #sents = random.shuffle(sents)
    res = []
    
    for sent in sents:
        save = False
        sentence_string = []
        #print(sent)
        #for item in sent:
            #print(item.word)
        for chunk in sent:
            sense = chunk.label()
            #print(sense)
            if isinstance(sense, str):
                None
            elif sense is None:
                None
            else:
              #  print(sense.name())
    #         sense = chunk.label
    #         print(sense)
    #         print(wordnet_sense)

    #         #print(sense.type())
                if sense.key() == wn_lemma.key():
                    print("found ", sense)
                    save = True
            #print("leaves")
            #print(chunk.leaves())
            sentence_string.append(chunk.leaves())

        if save==True:
            sentence_string = list(itertools.chain(*sentence_string))
            sentence_string = ' '.join(sentence_string)
            res.append(sentence_string)
    
    return res
        
sents = semcor.tagged_sents( tag = ' sem ' )

for sense in wordnet_senses:
    contexts = find_tokens(sense, sents)
    print(sense)
    print(contexts)

found  Lemma('bat.n.01.bat')
found  Lemma('bat.n.01.bat')
Lemma('bat.n.01.bat')
['Or the surging whirling sounds of bats at night , when their black bodies dived into the blackness above and below the amber street lights .', 'Out of the church and into his big car , it tooling over the road with him driving and the headlights sweeping the pike ahead and after he hit college , his expansiveness , the quaint little pine board tourist courts , cabins really , with a cute naked light bulb in the ceiling ( unfrosted and naked as a streetlight , like the one on the corner where you used to play when you were a kid , where you watched the bats swooping in after the bugs , watching in between your bouts at hopscotch ) , a room complete with moths pinging the light and the few casual cockroaches cruising the walls , an insect Highway Patrol with feelers waving .']
Lemma('mouse.n.04.mouse')
[]


In [173]:
### We really want to ensure these models are trained on everyhing but our eval words


# gold vectors, multiple sentences per word
"""
we want to take each feature cue / wordnet lemma pair and get the feature norm for the pair, 
as well as the sentences from semcor with that lemma

and just put them in a list of tuples
"""
data = []

# cue_words = []
# for w1, w2 in ambiguous_pairs:
#     cue_words.append(w1)
#     cue_words.append(w1)
# print(cue_words)

for cue_word, lemma in senses_to_analyze.items():
    print(cue_word)
    gold_vector = norms.get_feature_vector(cue_word)
    
    contexts = find_tokens(lemma, sents)
    print(contexts)
    
    for context in contexts:
        row = (cue_word, lemma, context)
        data.append(row)

"""

then we ge tthe token representation of the lemma in each sentence, and average them
"""

bat_animal
found  Lemma('bat.n.01.bat')
found  Lemma('bat.n.01.bat')
['Or the surging whirling sounds of bats at night , when their black bodies dived into the blackness above and below the amber street lights .', 'Out of the church and into his big car , it tooling over the road with him driving and the headlights sweeping the pike ahead and after he hit college , his expansiveness , the quaint little pine board tourist courts , cabins really , with a cute naked light bulb in the ceiling ( unfrosted and naked as a streetlight , like the one on the corner where you used to play when you were a kid , where you watched the bats swooping in after the bugs , watching in between your bouts at hopscotch ) , a room complete with moths pinging the light and the few casual cockroaches cruising the walls , an insect Highway Patrol with feelers waving .']
bat_baseball
[]
board_wood
found  Lemma('board.n.02.board')
found  Lemma('board.n.02.board')
found  Lemma('board.n.02.board')
found  Lemma('boa

[]
pipe_smoking
found  Lemma('pipe.n.01.pipe')
found  Lemma('pipe.n.01.pipe')
found  Lemma('pipe.n.01.pipe')
found  Lemma('pipe.n.01.pipe')
['He finished with the team and filled his pipe and stood looking about him .', 'He would order her to bring coffee , and would take from his vest pocket a thin black pipe which he would stuff - he would not remove his gloves - and light and smoke .', "`` Laura , what would you say if I smoked a pipe '' ?", 'And as for his pipe , if he wanted to smoke one , nobody would stop him .']
pipe_plumbing
found  Lemma('pipe.n.02.pipe')
found  Lemma('pipe.n.02.pipe')
found  Lemma('pipe.n.02.pipe')
found  Lemma('pipe.n.02.pipe')
['The effluent was collected through two pipes and discharged to the Blue River through a surface drainage ditch .', 'It consists of a series of pipes and a pressure measuring chamber which record the rise and fall of the water surface .', 'Vent pipes also are necessary ( as shown in figs. 9 , 10 , and 11 ) , but filters are not .', '

'\n\nthen we ge tthe token representation of the lemma in each sentence, and average them\n'

In [187]:
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr

"""
data is a list of tuples of (cue word, lemma, context)

for each model, we want to run an analysis of these
"""

def pluralize(string):
    if string == 'mouse':
        return 'mice'
    else:
        return string + 's'



#print(data)

def evaluate_in_context(model, data, bert):
    n = 0

    correlations = []
    cosines = []
    top_k_precs = []

    for cue_word, lemma, context in data:
        n +=1
        singular = lemma.name()
        plural = pluralize(singular)

        gold_vector = norms.get_feature_vector(singular)
        gold_feats = norms.get_features(singular)
        k = len(gold_feats)



        try:
            predicted_vector = model.predict_in_context(singular, context, bert)
            top_k =  model.predict_top_n_features_in_context(singular, context, k, bert)
        except:
            predicted_vector = model.predict_in_context(plural, context, bert)
            top_k =  model.predict_top_n_features_in_context(plural, context, k, bert)

        cos = cosine(predicted_vector, gold_vector)
        cosines.append(cos)      

        num_in_top_k = len(set(top_k).intersection(set(gold_feats)))
        top_k_prec = num_in_top_k / k
        top_k_precs.append(top_k_prec)

        corr, p = spearmanr(predicted_vector, gold_vector)
        correlations.append(corr)

        #print("cosine: %f" % cos)
        #print("precison: %f" % prec)
        #print("correlation: %f" % corr)
        #print("top k acc: %f" % top_k_prec)

    print("Average cosine between gold and predicted feature norms: %s" % np.average(cosines))
    #print("average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector: ", top_10_prec)
    #print("average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector: ", top_20_prec)
    print("Average % @k (derby metric)", np.average(top_k_precs))
    #print("Percentage (%) of test items that retrieve their gold-standard vector in the top 10 neighbours of their predicted vector: %f" % top_20_acc)
    print("correlation between gold and predicted vectors: %s " % np.average(correlations))

    print("total number of predictions: ", n)
    
single_model = torch.load('model.30eps.1k.saved')
multi_model = torch.load('model.30eps.5k.saved')
glove_model = torch.load('glove.model.60eps.saved')


bert = BERTBase()

evaluate_in_context(single_model, data, bert)

evaluate_in_context(multi_model, data, bert)


INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpe8wm7b26
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

Average cosine between gold and predicted feature norms: 0.7025733042366092
Average % @k (derby metric) 0.24348065371321184
correlation between gold and predicted vectors: 0.08068301476059604 
total number of predictions:  55
Average cosine between gold and predicted feature norms: 0.7072898472570438
Average % @k (derby metric) 0.22215577703949796
correlation between gold and predicted vectors: 0.07574852895168259 
total number of predictions:  55


In [191]:
from models import *

models = [single_model, multi_model, word2vec_model]

for model in models:
    
    exs = [lemma.name() for cue_word, lemma, context in data]
    evaluate(model, exs, norms, None, debug='info')


bat
['wood', 'human', 'insect', 'animal', 'long', 'metal', 'hand', 'bird', 'leg', 'fly']
['animal', 'cricket', 'hit', 'fly', 'wood']
['animal', 'cricket', 'fly', 'hit', 'wood']
['fly', 'animal', 'long', 'leg', 'hand']
cosine: 0.741780
precison: 0.600000
correlation: 0.057883
top k acc: 0.400000
mouse
['mammal', 'small', 'animal', 'fur', 'four', 'tail', 'long', 'brown', 'eat', 'leg']
['big', 'dig', 'fast', 'food', 'funny', 'inside', 'kill', 'live', 'outside', 'scare']
['animal', 'big', 'brown', 'cat', 'cheese', 'compute', 'cute', 'dig', 'ear', 'eat', 'experiment', 'fast', 'field', 'food', 'four', 'funny', 'fur', 'gray', 'grey', 'hair', 'house', 'inside', 'kill', 'leg', 'like', 'live', 'long', 'mammal', 'nose', 'outside', 'pet', 'rodent', 'scare', 'sharp', 'small', 'soft', 'squeak', 'steal', 'tail', 'trap', 'wall', 'whisker', 'white']
['lay', 'on', 'cat', 'dog', 'cute', 'feather', 'fast', 'funny', 'play', 'write', 'grey', 'sound', 'dirt', 'hard', 'wing', 'sharp', 'tree', 'eye', 'insect',